# Example of running sar-pre-processing package

## 1. Requirements

- Installation of sar-pre-processing package
- Installation of ESA's SNAP Toolbox (https://step.esa.int/main/download/snap-download/)
    - Currently only SNAP version 8.0 can be downloaded from the website. To update SNAP to a version >8.0.3 please start the SNAP software. You will be asked if you want to search for update. After the updates are installed you need to restart SNAP to initialize the installed updates.
    - SNAP Toolbox (Linux version) need libgfortran for specific operations but currently libgfortran is not installed during the installation process of SNAP therefore you might use
```sudo apt-get install gfortran```
- Sentinel-1 SLC data
    - location where files are stored (locally)<br>
    **or**
    - use 2. to download data from the internet
        - Free account for Copernicus Sentinel Data Hub needs to be created (https://scihub.copernicus.eu/dhus/#/self-registration)
        - free disk space needed
- For plotting purposes within this jupyter notebook matplotlib needs to be installed
    - ```conda install -c conda-forge matplotlib```

## 2. Download sample data from Sentinel Data Hub

### <span style="color:red">Create Account (https://scihub.copernicus.eu/dhus/#/self-registration) and change user and password below</span>.

In [ ]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
user = 'username'
password = 'password'
# initialize settings
api = SentinelAPI(user, password)

In [ ]:
# search by polygon (MNI test site coordinates), time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('coordinates_mni.geojson'))
products = api.query(footprint,
                     date=('20210601', '20210615'),
                     platformname='Sentinel-1',
                     producttype='SLC')
print('Following products will be downloaded')
print(api.to_dataframe(products).title.values)

print('These {} product need {} Gb of disk space'.format(len(products), api.get_products_size(products)))

**Start download process (<span style="color:red">Attention: might take a while and data will requries some free disk space</span>)**

In [ ]:
# download all results from the search
# files will be downloaded to specified path
import os
path = "/path/to/data/"
try:
    os.makedirs(path)
except: FileExistsError
api.download_all(products, path)

## 3. Use sar-pre-processing package to process Sentinel-1 SLC data

Set paths for
- input_folder (path to stored Sentinel-1 SLC data (zip files) e.g. "~/Downloads")
- output_folder (path where processed data will be stored e.g. "~/output")
- gpt_loction (gpt is located in the bin folder of your SNAP installation)' 

In [ ]:
input_folder = os.path.expanduser(path)
output_folder = os.path.expanduser(path)
gpt_location = os.path.expanduser('~/snap/bin/gpt')

Create config file with information about input, output and gpt loction

In [ ]:
import yaml

with open('sample_config_file.yaml') as stream:
   data = yaml.safe_load(stream)

data['input_folder'] = input_folder
data['output_folder'] = output_folder
data['gpt'] = gpt_location

with open('test_config_file.yaml', 'wb') as stream:
   yaml.safe_dump(data, stream, default_flow_style=False, 
                  explicit_start=True, allow_unicode=True, encoding='utf-8')

Optional config options which might be useful

In [ ]:
with open('test_config_file.yaml') as stream:
   data = yaml.safe_load(stream)

# Filter option
## Filter via year of interes
#data['year'] = '2021'

## Define region of interest
#data['region']['lr']['lat'] = 48.2 # lower right latitude
#data['region']['lr']['lon'] = 11.9 # lower right longitude
#data['region']['ul']['lat'] = 48.4 # upper left latitude
#data['region']['ul']['lon'] = 11.6 # upper left longitude
#data['region']['subset'] = 'yes'

## Define multi-temporal filtering properties
#data['speckle_filter']['multi_temporal']['apply'] = 'yes'
#data['speckle_filter']['multi_temporal']['files'] = '5' # Number of files used for multi temporal filtering

## Define incidence angle for normalization
#data['normalization_angle'] = '35'

## Processing of one single file instead of a time series
#data['speckle_filter']['multi_temporal']['apply'] = 'no'

with open('test_config_file.yaml', 'wb') as stream:
   yaml.safe_dump(data, stream, default_flow_style=False, 
                  explicit_start=True, allow_unicode=True, encoding='utf-8')

Start pre-processing steps

In [ ]:
from sar_pre_processing.sar_pre_processor import *
import warnings
warnings.filterwarnings("ignore")

processing = SARPreProcessor(config='test_config_file.yaml')
processing.create_processing_file_list()
print('start step 1')
processing.pre_process_step1()
print('start step 2')
processing.pre_process_step2()
print('start step 3')
processing.pre_process_step3()
print('start add netcdf information')
processing.add_netcdf_information()
print('start create netcdf stack')
processing.create_netcdf_stack()

## 3. View processed data

Load netcdf file with processed data

In [ ]:
import os
print(os.getcwd())
print(output_folder)

In [ ]:
from netCDF4 import Dataset
import numpy as np

my_example_nc_file = os.path.join(output_folder, 'data.nc')
data = Dataset(my_example_nc_file, mode='r') 

View information about dataset

In [ ]:
data

Read data from netcdf file

In [ ]:
data.variables['orbitdirection'][:]
data.variables['time'][:]
lons = data.variables['lon'][:]
lats = data.variables['lat'][:]
vv = data.variables['sigma0_vv_single'][:]

vv_units = data.variables['sigma0_vv_single'].units

Close netcdf file

In [ ]:
data.close()

Plot vv polorized data

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    # Problem: border pixel might be zero or negative
    # pixel eqal or smaller than zero are set to nan
    array = np.copy(vv[x])
    array[array <= 0] = np.nan
    # plot backscatter data in dB scale
    plt.imshow(10*np.log10(array))
    cbar = plt.colorbar()
    cbar.set_label('dB')
    plt.clim(-25, 0)

interactive_plot = interactive(f, x=(0,len(vv)-1))
interactive_plot